In [ ]:
import numpy as np
z = np.array([2.20,1.39,0.85])
print(z)
exp_z = np.exp(z)
print(exp_z)
c = exp_z / np.sum(exp_z)
print(c)
print(np.sum(c))

In [ ]:
def softmax(z):
    z = np.clip(z, -100, None)            
    exp_z = np.exp(z)
    print(exp_z)
    print(np.sum(exp_z, axis=1))
    return exp_z / np.sum(exp_z, axis=1).reshape(-1, 1)

In [ ]:
z = np.array([[2.20,1.39,0.85],
              [0.00,-1.39,-2.20]])
c = softmax(z)
print(c)
print( np.sum(c, axis=1))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [ ]:
class SingleLayer:    
    def __init__(self, learning_rate=0.1, l1=0, l2=0):
        self.w = None              
        self.b = None              
        self.losses = []           
        self.val_losses = []       
        self.w_history = []        
        self.lr = learning_rate    
        self.l1 = l1               
        self.l2 = l2               

    def forpass(self, x):
        z = np.dot(x, self.w) + self.b        
        return z

    def backprop(self, x, err):
        m = len(x)
        w_grad = np.dot(x.T, err) / m         
        b_grad = np.sum(err) / m              
        return w_grad, b_grad

    def activation(self, z):
        z = np.clip(z, -100, None)            
        a = 1 / (1 + np.exp(-z))              
        return a
        
    def fit(self, x, y, epochs=100, x_val=None, y_val=None):
        y = y.reshape(-1, 1)                  
        y_val = y_val.reshape(-1, 1)
        m = len(x)                            
        self.w = np.ones((x.shape[1], 1))     
        self.b = 0                            
        self.w_history.append(self.w.copy())  
        
        for i in range(epochs):
            z = self.forpass(x)               
            a = self.activation(z)            
            err = a - y                   
            
            w_grad, b_grad = self.backprop(x, err)
            w_grad += (self.l1 * np.sign(self.w) + self.l2 * self.w) / m
            
            self.w -= self.lr * w_grad
            self.b -= self.lr * b_grad
           
            self.w_history.append(self.w.copy())
            
            a = np.clip(a, 1e-10, 1-1e-10)
            
            loss = np.sum(-(y*np.log(a) + (1-y)*np.log(1-a)))
            self.losses.append((loss + self.reg_loss()) / m)
            
            self.update_val_loss(x_val, y_val)
    
    def predict(self, x):
        z = self.forpass(x)      
        return z > 0             
    
    def score(self, x, y):
        return np.mean(self.predict(x) == y.reshape(-1, 1))
    
    def reg_loss(self):
        return self.l1 * np.sum(np.abs(self.w)) + self.l2 / 2 * np.sum(self.w**2)
    
    def update_val_loss(self, x_val, y_val):
        z = self.forpass(x_val)            
        a = self.activation(z)             
        a = np.clip(a, 1e-10, 1-1e-10)     
        
        val_loss = np.sum(-(y_val*np.log(a) + (1-y_val)*np.log(1-a)))
        self.val_losses.append((val_loss + self.reg_loss()) / len(y_val))

In [ ]:
class DualLayer(SingleLayer):
    
    def __init__(self, units=10, learning_rate=0.1, l1=0, l2=0):
        self.units = units         
        self.w1 = None             
        self.b1 = None             
        self.w2 = None            
        self.b2 = None            
        self.a1 = None             
        self.losses = []          
        self.val_losses = []      
        self.lr = learning_rate   
        self.l1 = l1              
        self.l2 = l2             

    def forpass(self, x):
        z1 = np.dot(x, self.w1) + self.b1        
        self.a1 = self.activation(z1)            
        z2 = np.dot(self.a1, self.w2) + self.b2  
        return z2

    def backprop(self, x, err):
        m = len(x)       
        
        w2_grad = np.dot(self.a1.T, err) / m
        b2_grad = np.sum(err, axis=0) / m
        
        err_to_hidden = np.dot(err, self.w2.T) * self.a1 * (1 - self.a1)
       
        w1_grad = np.dot(x.T, err_to_hidden) / m
        b1_grad = np.sum(err_to_hidden, axis=0) / m
        return w1_grad, b1_grad, w2_grad, b2_grad

    def init_weights(self, n_features):
        self.w1 = np.ones((n_features, self.units))  
        self.b1 = np.zeros(self.units)               
        self.w2 = np.ones((self.units, 1))           
        self.b2 = 0
        
    def fit(self, x, y, epochs=100, x_val=None, y_val=None):
        y = y.reshape(-1, 1)          
        y_val = y_val.reshape(-1, 1)
        m = len(x)                    
        self.init_weights(x.shape[1]) 
        
        for i in range(epochs):
            a = self.training(x, y, m)
            a = np.clip(a, 1e-10, 1-1e-10)
            loss = np.sum(-(y*np.log(a) + (1-y)*np.log(1-a)))
            self.losses.append((loss + self.reg_loss()) / m)
            self.update_val_loss(x_val, y_val)
            
    def training(self, x, y, m):
        z = self.forpass(x)       
        a = self.activation(z)    
        err = (a - y)           
        
        w1_grad, b1_grad, w2_grad, b2_grad = self.backprop(x, err)
        w1_grad += (self.l1 * np.sign(self.w1) + self.l2 * self.w1) / m
        w2_grad += (self.l1 * np.sign(self.w2) + self.l2 * self.w2) / m

        self.w1 -= self.lr * w1_grad
        self.b1 -= self.lr * b1_grad

        self.w2 -= self.lr * w2_grad
        self.b2 -= self.lr * b2_grad
        return a
    
    def reg_loss(self):
        return self.l1 * (np.sum(np.abs(self.w1)) + np.sum(np.abs(self.w2))) + \
               self.l2 / 2 * (np.sum(self.w1**2) + np.sum(self.w2**2))

In [ ]:
class RandomInitNetwork(DualLayer):
    
    def init_weights(self, n_features):
        np.random.seed(42)
        self.w1 = np.random.normal(0, 1, 
                                   (n_features, self.units))  
        self.b1 = np.zeros(self.units)                      
        self.w2 = np.random.normal(0, 1, 
                                   (self.units, 1))          
        self.b2 = 0

In [ ]:
class MinibatchNetwork(RandomInitNetwork):
    
    def __init__(self, units=10, batch_size=32, learning_rate=0.1, l1=0, l2=0):
        super().__init__(units, learning_rate, l1, l2)
        self.batch_size = batch_size     
        
    def fit(self, x, y, epochs=100, x_val=None, y_val=None):
        y_val = y_val.reshape(-1, 1)     
        self.init_weights(x.shape[1])    
        
        for i in range(epochs):
            loss = 0
            
            for x_batch, y_batch in self.gen_batch(x, y):
                y_batch = y_batch.reshape(-1, 1) 
                m = len(x_batch)                 
                a = self.training(x_batch, y_batch, m)
                a = np.clip(a, 1e-10, 1-1e-10)
                loss += np.sum(-(y_batch*np.log(a) + (1-y_batch)*np.log(1-a)))

            self.losses.append((loss + self.reg_loss()) / len(x))
            self.update_val_loss(x_val, y_val)

    def gen_batch(self, x, y):
        length = len(x)
        bins = length // self.batch_size 
        bins += 1                    
        indexes = np.random.permutation(np.arange(len(x))) 
        x = x[indexes]
        y = y[indexes]
        for i in range(bins):
            start = self.batch_size * i
            end = self.batch_size * (i + 1)
            yield x[start:end], y[start:end]   

In [ ]:
class MultiClassNetwork(MinibatchNetwork):
    def __init__(self, units=10, batch_size=32, learning_rate=0.1, l1=0, l2=0):
        self.units = units         
        self.batch_size = batch_size     
        self.w1 = None             
        self.b1 = None             
        self.w2 = None             
        self.b2 = None             
        self.a1 = None             
        self.losses = []           
        self.val_losses = []       
        self.lr = learning_rate    
        self.l1 = l1               
        self.l2 = l2               

    def forpass(self, x):
        z1 = np.dot(x, self.w1) + self.b1        
        self.a1 = self.sigmoid(z1)               
        z2 = np.dot(self.a1, self.w2) + self.b2  
        return z2

    def backprop(self, x, err):
        m = len(x)       
        
        w2_grad = np.dot(self.a1.T, err) / m
        b2_grad = np.sum(err) / m
        
        err_to_hidden = np.dot(err, self.w2.T) * self.a1 * (1 - self.a1)
        
        w1_grad = np.dot(x.T, err_to_hidden) / m
        b1_grad = np.sum(err_to_hidden, axis=0) / m
        return w1_grad, b1_grad, w2_grad, b2_grad
    
    def sigmoid(self, z):
        z = np.clip(z, -100, None)            
        a = 1 / (1 + np.exp(-z))              
        return a
    
    def softmax(self, z):
        z = np.clip(z, -100, None)            
        exp_z = np.exp(z)
        return exp_z / np.sum(exp_z, axis=1).reshape(-1, 1)
 
    def init_weights(self, n_features, n_classes):
        self.w1 = np.random.normal(0, 1, 
                                   (n_features, self.units))  
        self.b1 = np.zeros(self.units)                        
        self.w2 = np.random.normal(0, 1, 
                                   (self.units, n_classes))   
        self.b2 = np.zeros(n_classes)
        
    def fit(self, x, y, epochs=100, x_val=None, y_val=None):
        np.random.seed(42)
        self.init_weights(x.shape[1], y.shape[1])    
        
        for i in range(epochs):
            loss = 0
            print('.', end='')
            for x_batch, y_batch in self.gen_batch(x, y):
                a = self.training(x_batch, y_batch)
                a = np.clip(a, 1e-10, 1-1e-10)
                loss += np.sum(-y_batch*np.log(a))
            self.losses.append((loss + self.reg_loss()) / len(x))
            self.update_val_loss(x_val, y_val)

    def gen_batch(self, x, y):
        length = len(x)
        bins = length // self.batch_size 
        if length % self.batch_size:
            bins += 1                    
        indexes = np.random.permutation(np.arange(len(x))) 
        x = x[indexes]
        y = y[indexes]
        for i in range(bins):
            start = self.batch_size * i
            end = self.batch_size * (i + 1)
            yield x[start:end], y[start:end]   
            
    def training(self, x, y):
        m = len(x)                
        z = self.forpass(x)       
        a = self.softmax(z)       
        err = (a - y)            
        
        w1_grad, b1_grad, w2_grad, b2_grad = self.backprop(x, err)
        w1_grad += (self.l1 * np.sign(self.w1) + self.l2 * self.w1) / m
        w2_grad += (self.l1 * np.sign(self.w2) + self.l2 * self.w2) / m

        self.w1 -= self.lr * w1_grad
        self.b1 -= self.lr * b1_grad

        self.w2 -= self.lr * w2_grad
        self.b2 -= self.lr * b2_grad
        return a
   
    def predict(self, x):
        z = self.forpass(x)          
        return np.argmax(z, axis=1)  
    
    def score(self, x, y):
        return np.mean(self.predict(x) == np.argmax(y, axis=1))

    def reg_loss(self):
        return self.l1 * (np.sum(np.abs(self.w1)) + np.sum(np.abs(self.w2))) + \
               self.l2 / 2 * (np.sum(self.w1**2) + np.sum(self.w2**2))

    def update_val_loss(self, x_val, y_val):
        z = self.forpass(x_val)            
        a = self.softmax(z)                
        a = np.clip(a, 1e-10, 1-1e-10)     
        
        val_loss = np.sum(-y_val*np.log(a))
        self.val_losses.append((val_loss + self.reg_loss()) / len(y_val))


In [ ]:
import tensorflow as tf
import keras
import matplotlib.pyplot as plt

(x_train_all, y_train_all), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
print(x_train_all.shape, y_train_all.shape)

In [ ]:
plt.imshow(x_train_all[8], cmap='gray')
plt.show()

In [ ]:
print(y_train_all[:100])

In [ ]:
class_names = ['티셔츠/윗도리', '바지', '스웨터', '드레스', '코트', 
               '샌들', '셔츠', '스니커즈', '가방', '앵클부츠']
print(class_names[y_train_all[8]])

#### np.bincount 예제

In [ ]:
a = np.array([0,4,4,1,2,3,3,3,3,4,5,0])
print(len(a))
print(a.shape[0])
print(np.bincount(a))

In [ ]:
print(len(y_train_all))
np.bincount(y_train_all)

In [ ]:
print(len(y_test))
np.bincount(y_test)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train_all, y_train_all, stratify=y_train_all, test_size=0.2, random_state=42)

In [ ]:
print(len(y_train))
np.bincount(y_train)

In [ ]:
print(len(y_val))
np.bincount(y_val)

In [ ]:
print(x_train[0])

In [ ]:
x_train = x_train / 255
x_val = x_val / 255

In [ ]:
print(x_train[0])

In [ ]:
print(x_train.shape)

In [ ]:
x_train = x_train.reshape(-1, 784)
x_val = x_val.reshape(-1, 784)
print(x_train.shape, x_val.shape)

In [ ]:
y_train[0]

In [ ]:
tf.keras.utils.to_categorical([0, 1, 3]) # 원 핫 인코딩

In [ ]:
y_train_encoded = tf.keras.utils.to_categorical(y_train)
y_val_encoded = tf.keras.utils.to_categorical(y_val)
print(y_train_encoded.shape, y_val_encoded.shape)

In [ ]:
print(y_train[0], y_train_encoded[0])

In [ ]:
fc = MultiClassNetwork(units=100, batch_size=256)
fc.fit(x_train, y_train_encoded, 
       x_val=x_val, y_val=y_val_encoded, epochs=40)
plt.plot(fc.losses)
plt.plot(fc.val_losses)
plt.ylabel('loss')
plt.xlabel('iteration')
plt.legend(['train_loss', 'val_loss'])
plt.show()

In [ ]:
x_test = x_test.reshape(-1, 784)

In [ ]:
y_test_encoded = tf.keras.utils.to_categorical(y_test)

In [ ]:
fc.score(x_test, y_test_encoded)